# NDArray API example

The NDArray API, defined in the ndarray (or simply nd) package, provides imperative tensor operations on CPU/GPU. An NDArray represents a multi-dimensional, fixed-size homogenous array.

In [ ]:
import logging
logging.basicConfig(level=logging.WARN)  # disable the verbose INFO messages for cleaner notebook display

In [ ]:
import mxnet as mx
import numpy as np

In [ ]:
sample_count = 1000
train_count = 800
valid_count = sample_count - train_count
feature_count = 100
category_count = 10
X = mx.nd.uniform(low=0, high=1, shape=(sample_count,feature_count))
Y = mx.nd.empty((sample_count,))

In [ ]:
for i in range(0, sample_count-1):
   Y[i] = np.random.randint(0, category_count)
X_train = mx.nd.crop(X, begin=(0,0), end=(train_count, feature_count))
Y_train = Y[0:train_count]
X_valid = mx.nd.crop(X, begin=(train_count, 0), end=(sample_count, feature_count))
Y_valid = Y[train_count:sample_count]
print(X.shape, Y.shape, X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape)

# SYmbol API example

The Symbol API, defined in the symbol (or simply sym) package, provides neural network graphs and auto-differentiation. A symbol represents a multi-output symbolic expression. They are composited by operators, such as simple matrix operations (e.g. “+”), or a neural network layer (e.g. convolution layer). An operator can take several input variables, produce more than one output variables, and have internal state variables. A variable can be either free, which we can bind with value later, or an output of another symbol.

In [ ]:
data = mx.sym.Variable('data')
fc1 = mx.sym.FullyConnected(data, name='fc1', num_hidden=64)
relu1 = mx.sym.Activation(fc1, name='relu1', act_type="relu")
fc2 = mx.sym.FullyConnected(relu1, name='fc2', num_hidden=category_count)
out = mx.sym.SoftmaxOutput(fc2, name='softmax')

# Iterator (Data Loading) API example

Let’s see how to write an iterator for a new data format. The following iterator can be used to train a symbol whose input data variable has name X_train and input label variable has name Y_train. The iterator also provides information about the batch, including the shapes and name.

In [ ]:
batch = 10
train_iter = mx.io.NDArrayIter(data=X_train, label=Y_train, batch_size=batch)
val_iter = mx.io.NDArrayIter(data=X_valid, label=Y_valid, batch_size=batch)

In [ ]:
print(train_iter.provide_data)

In [ ]:
print(train_iter.provide_label)

In [ ]:
print(val_iter.provide_data)

In [ ]:
print(val_iter.provide_label)

# Module API example

The module API, defined in the module (or simply mod) package, provides an intermediate and high-level interface for performing computation with a Symbol. One can roughly think a module is a machine which can execute a program defined by a Symbol.

In [ ]:
nb_epochs=50
mod = mx.mod.Module(out)

In [ ]:
mod.bind(data_shapes=train_iter.provide_data,
         label_shapes=train_iter.provide_label)
mod.init_params(initializer=mx.init.Xavier(magnitude=2.))
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 0.1), ))


# Train the module

In [ ]:
mod.fit(train_iter, num_epoch=nb_epochs)

metric = mx.metric.create('acc')
# train 5 epochs, i.e. going over the data iter one pass
for epoch in range(nb_epochs):
    train_iter.reset()
    metric.reset()
    for batch in train_iter:
        mod.forward(batch, is_train=True)       # compute predictions
        mod.update_metric(metric, batch.label)  # accumulate prediction accuracy
        mod.backward()                          # compute gradients
        mod.update()                            # update parameters
    print('Epoch %d, Training %s' % (epoch, metric.get()))

In [ ]:
mod.save_checkpoint("myFirstModel", nb_epochs)